In [8]:
import tensorflow as tf
from numpy.random import sample
from tensorflow import keras, losses
from tensorflow.keras import datasets, layers, models
# import matplotlib.pyplot as plt

(X,y),(x_test,y_test) = datasets.mnist.load_data()
X = tf.convert_to_tensor(X,dtype=tf.float32) / 255.
y = tf.convert_to_tensor(y,dtype=tf.int32)

x_test = tf.convert_to_tensor(x_test,dtype=tf.float32) / 255.
y_test = tf.convert_to_tensor(y_test,dtype=tf.int32)
train_db = tf.data.Dataset.from_tensor_slices((X, y)).batch(128)

test_db = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(128)
w1 = tf.Variable(tf.random.truncated_normal(shape=(784,256), dtype=tf.float32, stddev=0.1))
b1 = tf.Variable(tf.zeros(256))
w2 = tf.Variable(tf.random.truncated_normal(shape=(256,128), dtype=tf.float32, stddev=0.1))
b2 = tf.Variable(tf.zeros(128))
w3 = tf.Variable(tf.random.truncated_normal(shape=(128,10), dtype=tf.float32, stddev=0.1))
b3 = tf.Variable(tf.zeros(10))
lr = 0.001
def model():
    for step,(x,y) in enumerate(train_db):
        """
        x [128,28,28]
        y [128]
        """
        x = tf.reshape(x, (-1,784))


        with tf.GradientTape() as tape:
            h1 = x @ w1 + b1
            h1 = tf.nn.relu(h1)
            h2 = h1 @ w2 + b2
            h2 = tf.nn.relu(h2)
            out = h2 @ w3 + b3

            y_one_hot = tf.one_hot(y, depth=10)

            loss = tf.square(y_one_hot - out)
            loss = tf.reduce_mean(loss)

        grads = tape.gradient(loss, [w1, b1, w2, b2, w3, b3])
        w1.assign_sub(lr * grads[0])
        b1.assign_sub(lr * grads[1])
        w2.assign_sub(lr * grads[2])
        b2.assign_sub(lr * grads[3])
        w3.assign_sub(lr * grads[4])
        b3.assign_sub(lr * grads[5])
        if step % 100 == 0:
            print("loss",step,float(loss))
for i in range(10):
    print(i)
    model()
    corrects,total = 0,0
    for step,(x,y) in enumerate(test_db):
        x = tf.reshape(x, (-1,784))
        h1 = x @ w1 + b1
        h1 = tf.nn.relu(h1)
        h2 = h1 @ w2 + b2
        h2 = tf.nn.relu(h2)
        out = h2 @ w3 + b3

        prob = tf.nn.softmax(out,axis=-1)
        pred = tf.cast(tf.argmax(prob,axis=-1),dtype=tf.int32)
        correct = tf.reduce_sum(tf.cast(tf.equal(pred,y),dtype=tf.int32))
        corrects += int(correct)
        total += x.shape[0]
    print("Accuracy:",corrects / total)



0
loss 0 0.6823737621307373
loss 100 0.24574828147888184
loss 200 0.188330739736557


KeyboardInterrupt: 

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# 创建一个简单的神经网络模型
def build_model(activation_function):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, input_shape=(784,), activation=activation_function),  # 隐藏层
        tf.keras.layers.Dense(10, activation='softmax')  # 输出层，使用 Softmax 激活函数
    ])
    return model

# 定义激活函数
activation_functions = {
    'Sigmoid': tf.keras.activations.sigmoid,
    'Tanh': tf.keras.activations.tanh,
    'ReLU': tf.keras.activations.relu,
    'Leaky ReLU': tf.keras.layers.LeakyReLU(alpha=0.01)
}

# 可视化激活函数
def plot_activation_functions():
    x = np.linspace(-5, 5, 100)
    plt.figure(figsize=(12, 8))

    # Sigmoid
    plt.subplot(2, 2, 1)
    plt.plot(x, tf.keras.activations.sigmoid(x), label='Sigmoid')
    plt.title('Sigmoid')
    plt.grid(True)

    # Tanh
    plt.subplot(2, 2, 2)
    plt.plot(x, tf.keras.activations.tanh(x), label='Tanh')
    plt.title('Tanh')
    plt.grid(True)

    # ReLU
    plt.subplot(2, 2, 3)
    plt.plot(x, tf.keras.activations.relu(x), label='ReLU')
    plt.title('ReLU')
    plt.grid(True)

    # Leaky ReLU
    plt.subplot(2, 2, 4)
    leaky_relu = tf.keras.layers.LeakyReLU(alpha=0.01)
    plt.plot(x, leaky_relu(x), label='Leaky ReLU')
    plt.title('Leaky ReLU')
    plt.grid(True)

    plt.tight_layout()
    plt.show()

# 可视化激活函数
plot_activation_functions()

# 创建并编译模型
for name, activation in activation_functions.items():
    print(f"Building model with {name} activation function.")
    model = build_model(activation)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    print("\n")